# The PYCV module for PLUMED 2 Tutorial
## Objective  
This tutorial guides you step by step through setting up and using Pycv interface for PLUMED. It allows users to define custom collective variables in Pyhton and interface them with PLUMED's engine. No c++ coding is required.

By the end of this tutorial, you will be able to:
* Install PLUMED with Pycv support.
* Define Differentiable CVs in Python.
* Link the CV to plumed.
* Run the CV through the plumed Driver.


## Instalation 
### Prerequisites  
The following installation is based a MacOS. 
The virtual environment is set using python3 venv.

### Installation  
Create a working directory, for example, `/my_directory` then 
1. Clone the plumed source code (version 2.10)  

    `git clone --branch 2.10 https://github.com/plumed/plumed2.git`  
    
At the end of the clonning, you shall find a `plumed2` directory  

2. Create and activate a Python environment:  
for example  

    `pyhton3 -m venv pycvenv`  
    `source pycvenv/bin/activate`  

3. Configuration  
Make sure you are in the  `plumed2` directory  

    `./configure`  

4. Compilation  

    `make -j4`  

5. Source the PLUMED environment script  
This step sets up environment variables needed to use PLUMED  
    `source sourceme.sh`  

6. Install Python dependenciies and the PyCV interface.  
Navigate to the plugin directory  

    `cd plugin/pycv`  
    `pip install -r requirements.txt`  
    `pip install .`  

This concludes the installation procedure. 

## Uasge  
When wanting to use your PyCV enabled PLUMED, the environment variables explained below must be set in the same shell session where you execute the PLUMED driver.  
This is typically the terminal where your Python CV file and PLUMED input file reside and where you run the plumed driver command.  
Otherwise, PLUMED will not find the shared library or your Python module.  

1. Activate your virtual environment  
    `source /../../my_directory/pycvenv/bin/activate`  

2. Source the PLUMED shell environment  
    `source /../../my_directory/plumed2/sourceme.sh`  

3. Export your virtual environemnt `site-packages` to `PYTHONPATH`:  
Find the virtual environment `site-packages` path (example below uses Pyhton 3.10):  
    `export PYTHONPATH=$PYTHONPATH:/../../my_directory/pycvenv/lib/python3.10/site-packages`  

4. Locate the shared dynamic library `PythonCVInterface`:  
This will be used in the plumed input file for plumed to interpret and execute the Pyhton based CV.     



 




## Application Example  

In this example, we create a CV that calculates the distance between two atoms of our choice, say the first and second of our structure.   
To do so, the setup requires two files.  
1. A plumed input file `plumed_input.dat`.  
2. A python fille `python_file.py`.  

### plumed_input.dat  
`   LOAD FILE=/../PythonCVInterface.dylib  
    
    ca_atoms: ATOMS=1,2  

    d: PYCVINTERFACE ATOMS=ca_atoms IMPORT=distance_module CALCULATE=distance_fn   

    DUMPDERIVATIVES ARG=d FILE=cv_der.out  
    
    PRINT ARG=d FILE=cv.out STRIDE=1  
`

The work flow begins with the `LOAD` action, which loads the shared dynamic library that we located in the Usage section point 4.  
This links PLUMED to the Python interface.
  
    ` LOAD FILE=/.../PythonCVInterface.dylib `
    

Next, a label like  `ca\_atoms` is assigned to the atom group of interest using the `ATOMS` action. For example 
    ` ca_atoms: ATOMS=1,2`  
    There is more on atom selection in the PLUMED page [here](https://www.plumed.org/doc-v2.9/user-doc/html/_group.html#:~:text=Specifying%20Atoms,%2C20%2C10%20LABEL=g4).  
    
    
The line below tells PLUMED to look for a python file called `python_file.py`, import it then calculate the function `distance\_fn()` on the atoms defined by `ca\_atoms}`  
`d` at the beginning of the line is the label given to our CV.

    ` d: PYCVINTERFACE ATOMS=ca_atoms IMPORT=pyhton_file CALCULATE=distance_fn `
    
To print out the values of the derivatives of your CV with respect to the atomic coordinates; 

    ` DUMPDERIVATIVES ARG=d FILE=distance_der.out `  

### python_file.py  

`   
    import jax  

    import jax.numpy as jnp  

    from jax import grad, jit  

    import plumedCommunications as PLMD   
    
    jax.config.update("jax_enable_x64", True)  # To allow for double precision

    plumedInit = {
        "COMPONENTS": {
            "Distance": {"period": None, "derivative": True},}}
    
    @jit
    def distance(positions):
        dist = jnp.linalg.norm(positions[0]-positions[1])
        return dist

    grad_distance = grad(distance)
    dummy_box = jnp.zeros((3,3))
    
    def distance_fn(action: PLMD.PythonCVInterface):
        positions= action.getPositions()
        d = distance(positions)
        grad_d = grad_distance(positions)
        
        return d,grad_d,dummy_box  

`
* ` import plumedCommunications as PLMD ` is the python module provided by the PyCV plugin. It resides in your python environment `site-packages`. (Usage section, point3).  

* ` plumedInit = {`
        `"COMPONENTS": {`
           ` "Distance": {"period": None, "derivative": True},}} `  
This initialization dictionary is to tell plumed what are the prperties of our cv interms of periodicity and if it has a derivative or no. If for example `derivative` is set to false, then `DUMPDERIVATIVES ARG=d FILE=cv_der.out ` in the plumed_input file will have nothing to write in `cv_der.out` file!  

* `@jit`
    `def distance(positions):`
        `dist = jnp.linalg.norm(positions[0]-positions[1])`
        `return dist`

   `grad_distance = grad(distance)`
   `dummy_box = jnp.zeros((3,3))`  
The `@jit` imported at the begining of the pyhton code `from jax import grad, jit ` is what transforms the python function into a compiled version for faster execution.  
`grad`. Check the [JAX page](https://docs.jax.dev/en/latest/_autosummary/jax.grad.html) for more details.  

    The `dummy_box` is a placeholder for the periodic box matrix that PLUMED expects, if our atomic positions are not within a boundary condition, then we set the box empty as in the example above.  `dummy_box = jnp.zeros((3,3))`.   

* `def distance_fn(action: PLMD.PythonCVInterface):`
        `positions= action.getPositions()`
        `d = distance(positions)`
        `grad_d = grad_distance(positions)`
        
        `return d,grad_d,dummy_box`  

    `PLMD.PythonCVInterface` is the interface class that plumed uses to pass positions into python as `positions= action.getPositions()` and receive from python the CV values, it's gradients and the box matrix `return d,grad_d,dummy_box`  

## Running PLUMED with PyCV from the terminal  
Once the python and the plumed input files are ready, we can run the PLUMED Driver command:  

`plumed driver --plumed plumed_input.dat --pdb /path/to/file.pdb --mf_dcd /path/to/file.dcd`  

For reproducibility and automation, you can use the following pyhton template  

    `import subprocess`

    `# Define parameters`
    `plumed_file = 'plumed_input.dat'`
    `pdbfile = '/path/to/file.pdb'`
    `dcdfile = '/path/to/file.dcd'`


    `# PLUMED command`
   ` command = [`
    `'plumed', 'driver',`
    `'--plumed', plumed_file,`
    `'--pdb', pdbfile,`
    '--mf_dcd', dcdfile`
    `]`

    `# Executing`
    `print(f"Running command: {' '.join(command)}")`
    `subprocess.run(command)`  

And run it with `pyhton python_template.py` throught he terminal.